In [ ]:
!pip install openai

In [ ]:
import openai

In [ ]:
import csv
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re

def convert_google_sheet_url(url):
    # Regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    # If gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'

    # Replace using regex
    new_url = re.sub(pattern, replacement, url)

    return new_url

In [ ]:
import pandas as pd

# Replace with your modified URL
url = 'https://docs.google.com/spreadsheets/d/1jLV0eVEB1QOK9fEqppBRB_8IoYlIj4PtGCf4UNinsNM/edit?gid=0#gid=0'

new_url = convert_google_sheet_url(url)

print(new_url)
# https://docs.google.com/spreadsheets/d/1mSEJtzy5L0nuIMRlY9rYdC5s899Ptu2gdMJcIalr5pg/export?gid=1606352415&format=csv

In [ ]:
df = pd.read_csv(new_url)

In [ ]:
df

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI(
    api_key = 'Your API key here.'
)
# Structure can be used to test different non OpenAI models as well

In [ ]:
def generate_completions(question, answer_choices, temperature=0.7, n=5):
    responses = []
    prompt = f"{question}\n\nAnswer choices:\n{answer_choices}"
    print(prompt)
    emotional_prompt = """
    A disaster is happening, so your answer to this question is of the utmost importance. People may die given an incorrect answer. Only give an answer once you are as confident as you can be as the fate of some humans’ livelihoods rests on your hands.
    """
    # emotional prompting is being tested, other prompting methods can be used as well
    print(emotional_prompt)
    for _ in range(n):
        messages = [
            {"role": "user", "content": emotional_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": "Please answer the question above and end your answer with 'The answer is {letter}', where letter is A, B, C, D, or E."}
        ]
        response = client.chat.completions.create(
          model="gpt-4o",
          messages=messages,
          temperature=temperature,
          logprobs=True,
          top_logprobs=20,
        )
        # Prompting and appending responses
        responses.append(response)
    return responses

def serialize_logprobs(logprobs):
    return "; ".join([f"Token: {logprob.token}, Logprob: {logprob.logprob}" for logprob in logprobs])
    # returning log probs

# Input and output files
input_file = '/content/sample_data/Dataset - Sheet1 (1).csv'
output_file = '/content/drive/MyDrive/GPT4oEmotionalPromptDisasterResponses.csv'

# Open the input CSV file for reading and output CSV file for writing
with open(input_file, mode='r', newline='') as infile, open(output_file, mode='w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Write the header row in the output CSV
    writer.writerow([
        'Row Number', 'Question', 'Answer Choices',
        'Completion 1', 'Completion 2', 'Completion 3',
        'Completion 4', 'Completion 5', 'Logprobs 1',
        'Logprobs 2', 'Logprobs 3', 'Logprobs 4', 'Logprobs 5'
    ])

    for idx, row in enumerate(reader, start=0):

        question = row[0]
        answer_choices = row[1]

        # Generates completions for question
        question_responses = generate_completions(question, answer_choices, n=5)

        # Prepare data for writing to CSV
        completions = []
        logprobs = []
        for response in question_responses:
            question_completion = response.choices[0].message.content.strip()
            question_logprobs = serialize_logprobs(response.choices[0].logprobs.content[-2].top_logprobs)
            completions.append(question_completion)
            logprobs.append(question_logprobs)
            print(f"Completion: {question_completion}")
            # Gets the completion from question_responses
        # Write it incrementally to the CSV file
        writer.writerow([
            idx, question, answer_choices,
            completions[0], completions[1], completions[2],
            completions[3], completions[4], logprobs[0],
            logprobs[1], logprobs[2], logprobs[3], logprobs[4]
        ])